In [6]:
import os
os.environ.setdefault('AWS_PROFILE', 'hfeneldevcloud')

'hfeneldevcloud'

In [7]:
!aws s3 ls

2023-01-29 05:54:55 aws-sam-cli-managed-default-samclisourcebucket-1l7fajkgdrgq8
2023-02-02 13:31:27 codepipeline-us-east-1-816738919204
2023-02-02 14:48:52 devcloud-lambda-pipeline
2023-01-24 04:25:03 hfenelaigithub
2023-03-04 22:31:26 hfenelcloud-transvideo-bucket
2023-03-04 21:46:41 serverless.metrics-firehose-backup-szwyhupv3lcfnegzynnppn


In [ ]:
!aws s3 rm s3://hfenelaigithub/ --recursive

In [ ]:
!aws s3 mb s3://hfenelaigithub

In [8]:
import boto3
#from boto3 import boto3 as bt

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('jobs')
table.delete_item(Key={'job_id': 'hfeneldata_ingest'})
item = {
    'job_id': 'hfeneldata_ingest',
    'job_description': 'Ingest hfenel cloud data to S3',
    'is_active': 'Y',
    'baseline_days': 3
}
table.put_item(Item=item)

{'ResponseMetadata': {'RequestId': 'BU9PM9ED9O30C647915C6O707VVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 03 Apr 2023 05:08:43 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'BU9PM9ED9O30C647915C6O707VVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [9]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('jobs')

baseline_days = table.get_item(Key={'job_id':'hfeneldata_ingest'})['Item']['baseline_days']

In [10]:
baseline_days

Decimal('3')

In [ ]:
from decimal import Decimal


Decimal('3')

In [11]:
int(baseline_days)

3

In [26]:
from datetime import datetime as dt
from datetime import timedelta as td

next_file = f'{dt.strftime(dt.now().date() - td(days=int(baseline_days)), "%Y-%m-%d")}-0.json.gz'

In [27]:
next_file

'2023-03-31-0.json.gz'

In [28]:
import requests
res = requests.get(f'https://data.gharchive.org/{next_file}')

In [29]:
s3_client =  boto3.client('s3')

In [30]:
type(s3_client)

botocore.client.S3

In [ ]:
s3_client.put_object?

In [31]:
bucket_name='hfenelaigithub'

In [32]:
type(res.content)

bytes

In [33]:
upload_res = s3_client.put_object(
    Bucket = bucket_name,
    Key = f'landing/hfactivity/{next_file}',
    Body = res.content
)

In [34]:
upload_res

{'ResponseMetadata': {'RequestId': '043NCJ02F2FTH534',
  'HostId': '/mbD9idCxn6OzqudzvMUfK+YAwJ0+xjKb9hNu+yYns9gFlIpt3ZMyHqVFcZsWCzr3UxKHAiwcYs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/mbD9idCxn6OzqudzvMUfK+YAwJ0+xjKb9hNu+yYns9gFlIpt3ZMyHqVFcZsWCzr3UxKHAiwcYs=',
   'x-amz-request-id': '043NCJ02F2FTH534',
   'date': 'Mon, 03 Apr 2023 05:24:20 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"ca1b56398925b662ca11774e427182b3"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"ca1b56398925b662ca11774e427182b3"',
 'ServerSideEncryption': 'AES256'}

In [35]:
!aws s3 ls s3://hfenelaigithub/landing/hfactivity

                           PRE hfactivity/


In [36]:
import pandas as pd

In [ ]:
#obj = s3_client.get_object(Bucket=f's3://{bucket_name}/landing/hfactivity/', Key=next_file)
df = pd.read_json(
    f's3://{bucket_name}/landing/hfactivity/{next_file}',
    lines=True,
    orient='records'
)

In [ ]:
df.shape

In [ ]:
df.count()

In [ ]:
df